In [2]:
import keras
from keras.models import Sequential
from keras.layers import Activation
from keras.layers.core import Dense
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy

In [20]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import json
import requests
import pickle

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

In [44]:
df_train=pd.read_csv("data/train_tweet.csv",index_col=False)
df_test=pd.read_csv("data/test_tweet.csv",index_col=False)
print(len(df_train),len(df_test))

27481 3534


In [45]:
df_train=df_train[['textID', 'text', 'sentiment']]
df=df_train.append(df_test)
len(df)
df=df.dropna()
df=df[['text', 'sentiment']]

In [46]:
df["sentiment"]=df["sentiment"].astype('category')
df["sentiment_target"]=df["sentiment"].cat.codes

In [47]:
df.head()

,text,sentiment,sentiment_target
0,"I`d have responded, if I were going",neutral,1
1,Sooo SAD I will miss you here in San Diego!!!,negative,0
2,my boss is bullying me...,negative,0
3,what interview! leave me alone,negative,0
4,"Sons of ****, why couldn`t they put them on t...",negative,0


In [48]:
x=df["text"].values
y=df["sentiment_target"].values
print(len(x),len(y))

31014 31014


In [49]:
sentences_train,sentences_test,y_train,y_test=train_test_split(x,y,test_size=0.8,random_state =0)
vectorizer = TfidfVectorizer(stop_words="english", max_features=1000, decode_error="ignore")
vectorizer.fit(sentences_train)

X_train = vectorizer.transform(sentences_train)
X_test  = vectorizer.transform(sentences_test)

In [50]:
X_train.shape

(6202, 1000)

In [54]:
#model
input_dim = X_train.shape[1]  # Number of features

model = Sequential()
model.add(layers.Dense(10, input_dim=input_dim, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [55]:
model.compile(loss='binary_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])
model.summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_19 (Dense)             (None, 10)                10010     
_________________________________________________________________
dense_20 (Dense)             (None, 1)                 11        
Total params: 10,021
Trainable params: 10,021
Non-trainable params: 0
_________________________________________________________________


In [58]:
history = model.fit(X_train, y_train,
                    epochs=10,
                    verbose=True,
                    validation_data=(X_test, y_test),
                    batch_size=30)

Train on 6202 samples, validate on 24812 samples
Epoch 1/10
6202/6202 [==============================] - 1s 156us/step - loss: -3.5579 - acc: 0.5240 - val_loss: -2.9080 - val_acc: 0.4902
Epoch 2/10
6202/6202 [==============================] - 1s 158us/step - loss: -3.5758 - acc: 0.5250 - val_loss: -2.9040 - val_acc: 0.4907
Epoch 3/10
6202/6202 [==============================] - 1s 158us/step - loss: -3.5904 - acc: 0.5274 - val_loss: -2.8988 - val_acc: 0.4904
Epoch 4/10
6202/6202 [==============================] - 1s 158us/step - loss: -3.6053 - acc: 0.5268 - val_loss: -2.8916 - val_acc: 0.4910
Epoch 5/10
6202/6202 [==============================] - 1s 156us/step - loss: -3.6214 - acc: 0.5293 - val_loss: -2.8887 - val_acc: 0.4901
Epoch 6/10
6202/6202 [==============================] - 1s 157us/step - loss: -3.6311 - acc: 0.5282 - val_loss: -2.8817 - val_acc: 0.4902
Epoch 7/10
6202/6202 [==============================] - 1s 163us/step - loss: -3.6426 - acc: 0.5297 - val_loss: -2.8717 - v

In [36]:
loss, accuracy = model.evaluate(X_train, y_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(X_test, y_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))

Training Accuracy: 0.5508
Testing Accuracy:  0.4833


In [40]:
plot_history(history)